# Compte rendu TP2 Info 834
## Simonet Théo

Les Imports nécéssaires :

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import matplotlib.pyplot as plt

On utilise pandas pour récupérer les données des fichiers .csv puis on tranforme ces dataframes en tables 

In [2]:
data_academies = pd.read_csv("academies_virgule.csv")
data_villes = pd.read_csv("villes_virgule.csv")

### Question 1

On a créé des fonctions pour convertir des dataframes en tables et inversement 

In [3]:
def df_to_table(df):
    return pa.Table.from_pandas(df)

def table_to_df(table):
    return table.to_pandas()

table_academies = df_to_table(data_academies)
table_villes = df_to_table(data_villes)

On créé des fonctions pour convertir des tables en fichiers parquets et in versement

In [5]:
def table_to_parquet(table, nom_fichier):
    return pq.write_table(table, '{}'.format(nom_fichier))

def parquet_to_table(document):
    return pq.read_table(document)

table_to_parquet(table_academies, 'parquet_academies')
table_to_parquet(table_villes, 'parquet_villes')

### Question 2

On crée une fonction pour récupérer le schemas d'une table

In [15]:
def get_schema(table):
    return table.schema

print(get_schema(table_villes))

dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1596


### Question 3

On créé une fonction pour recuperer la colonne d'une table par son nom 

In [16]:
def get_column(table, column_name):
    return table[column_name]

print(get_column(table_villes, "nom"))

[
  [
    "Ozan",
    "Cormoranche-sur-Saône",
    "Plagne",
    "Tossiat",
    "Pouillat",
    ...
    "Sada",
    "Tsingoni",
    "Saint-Barthélemy",
    "Saint-Martin",
    "Saint-Pierre-et-Miquelon"
  ]
]


### Question 4

On créé les fonctions : count, count_distinct, sum, min, max qui renvoient des statistiques sur une
colonne de table

In [18]:
def count(table, column):
    return pc.count(get_column(table, column))

def count_distinct(table, column):
    return pc.count_distinct(get_column(table, column))

def sum(table, column):
    return pc.sum(get_column(table, column))

def min(table, column):
    return pc.min(get_column(table, column))


def max(table, column):
    return pc.max(get_column(table, column))

print(count(table_villes, "nom"))
print(count_distinct(table_villes, "nom"))
print(sum(table_villes, "nb_hab_2012"))
print(min(table_villes, "nb_hab_2012"))
print(max(table_villes, "nb_hab_2012"))

36700
34034
64264646
0
2211000


### Question 5

On créé les fonctions filter_villes et filter_departements pour sélectionner les informations sur une ville (e.g. Annecy) et pour sélectionner les informations sur un département (e.g. Haute-Savoie) par ordre alphabétique des villes

In [20]:
def filter_ville(table, nom_ville):
    df = table_to_df(table)
    mask = df['nom'] == nom_ville
    return table.filter(mask.to_list())

def filter_departement(table, num_departement):
    df = table_to_df(table)
    mask = df['dep'] == num_departement
    return table.filter(mask.to_list()).sort_by("nom")

print(filter_ville(table_villes, "Annecy"))
print(filter_departement(table_villes, "74"))

pyarrow.Table
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
----
dep: [["74"]]
nom: [["Annecy"]]
cp: [["74000"]]
nb_hab_2010: [[50379]]
nb_hab_1999: [[50324]]
nb_hab_2012: [[50100]]
dens: [[3690]]
surf: [[13.65]]
long: [[6.11667]]
lat: [[45.9]]
...
pyarrow.Table
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
----
dep: [["74","74","74","74","74",...,"74","74","74","74","74"]]
nom: [["Abondance","Alby-sur-Chéran","Alex","Allinges","Allonzier-la-Caille",...,"Épagny","Étercy","Étrembières","Évian-les-Bains","Évires"]]
cp: [["74360","74540","74290","74200","74350",...,"74330","74150","74100","74500","74570"]]
nb_hab_2010: [[1342,2012,985,3982,1637,...,3899,698,1887,8142,1294]]
nb_hab_1999: [[1297,1631,791,3022,1080,...,3233,630,1429,7278,10

### Question 6

On créé la fonction pour calculer le nombre moyen d'habitants en 2012 pour un departement; on itere cette fonction sur les liste des numéro de départements

In [23]:
def nb_moy_hab(num_dep):
    table = filter_departement(table_villes, num_dep)
    nb_habitants_moy_2012 = sum(table, "nb_hab_2012").as_py()/count(table, "nom").as_py()
    return nb_habitants_moy_2012

print(nb_moy_hab("74"))

'''
for num_dep in get_column(table_villes, "dep").to_list():
    print("le nombre moyen d'habitants dans le {} est de {}".format(num_dep, nb_moy_hab(num_dep)))
'''

2437.074829931973


'\nfor num_dep in get_column(table_villes, "dep").to_list():\n    print("le nombre moyen d\'habitants dans le {} est de {}".format(num_dep, nb_moy_hab(num_dep)))\n'

### Question 7 

Opérations ensemblistes jointures : afficher les zones de vacances des villes ; les villes de la zone de
vacances A ; les départements des zones de vacances A et B ; le nombre de villes par académie ; 

### Question 8

Affichage avec matplotlib : e.g. histogramme de la distribution du nombre de villes par académie